In [ ]:
import numpy as np
import tensorflow as tf
import os

from triplet_dataset_loader import *
from TL_class import SiameseTripletModel
from TL_siamese_network import generate_siamese_triplet_network
from top_accuracy_functions import *

database_path = "../databases/my-fruit-recognition-small"
base_test_folder = "../databases/my-fruit-recognition-small"
split_ratio = (0.8, 0.1, 0.1)
image_size = (100,100)
batch_size = 32

auto = tf.data.AUTOTUNE

learning_rate = 0.0001
steps_per_epoch = 50
validation_steps = 10
epochs = 5

train_dataset, val_dataset, test_dataset = create_dataset(database_path, split_ratio, image_size, batch_size)

triplet_siamese_network = generate_siamese_triplet_network(image_size)

triplet_siamese_model = SiameseTripletModel(triplet_siamese_network)

triplet_siamese_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate))
triplet_siamese_model.summary()

triplet_siamese_model.load_weights("5epoch_model.weights.h5")

In [ ]:
embedding_model = triplet_siamese_network.get_layer("Embedding")

# Check if the embedding model is correctly extracted
# print(embedding_model.summary())

In [ ]:
mapF = MapFunction(image_size)

# Initialize an empty dictionary to store first image embeddings with labels
first_embeddings_dict = {}

# Loop through each subfolder in the base_test_folder
for root, dirs, files in os.walk(base_test_folder):
  # Skip hidden folders (".DS_Store")
  files = [f for f in files if f != ".DS_Store"]

  # Check if there are files in the subfolder
  if files:
    files_sorted = sorted(files)
    first_image_path = os.path.join(root, files_sorted[0])

    # Get relative path from base_test_folder
    relative_path = os.path.relpath(root, base_test_folder)

    preprocessed_image = mapF.decode_and_resize(first_image_path)
    image_tensor = tf.expand_dims(preprocessed_image, axis=0)
    embedding = embedding_model.predict(image_tensor)

    # Convert the embedding to a hashable type (e.g., tuple or string)
    hashable_embedding = tuple(embedding.flatten())  # Convert to a tuple

    # Store the hashable embedding and relative path label in the dictionary
    first_embeddings_dict[hashable_embedding] = relative_path

print("Number of First Embeddings:", len(first_embeddings_dict))

In [ ]:
print(type(list(list(first_embeddings_dict.keys())[0])))
print(first_embeddings_dict.values())

In [ ]:
# Initialize an empty dictionary to store test embeddings with labels
test_embeddings_dict = {}

# Generate embeddings for the test images and pair them with labels
for image_path, label in test_dataset.items():
    # Preprocess the image using the MapFunction
    preprocessed_image = mapF.decode_and_resize(image_path)

    # Convert the preprocessed image to a tensor and add batch dimension
    image_tensor = tf.expand_dims(preprocessed_image, axis=0)

    # Generate embedding for the image using the embedding model
    embedding = embedding_model.predict(image_tensor)

    # Convert embedding numpy array to tuple for hashable key
    embedding_tuple = tuple(embedding.flatten())

    # Store the embedding tuple and label directly in the dictionary
    test_embeddings_dict[embedding_tuple] = label

# Print the number of test embeddings generated
print("Number of Test Embeddings:", len(test_embeddings_dict))

In [ ]:
print(test_embeddings_dict.values())

In [ ]:
def calculate_top3_accuracy(test_data, reference_data):
    total_tests = len(test_data)
    correct_top3_count = 0

    for test_embedding, test_label in test_data.items():
        test_embedding_np = np.array(test_embedding)  # Convert tuple back to numpy array
        closest_3_embeddings = top3_accuracy(test_embedding_np, list(reference_data.keys()))

        closest_3_labels = [reference_data[embedding] for embedding in closest_3_embeddings]

        # print(test_label)
        # for label in closest_3_labels:
        #     print(label)
        # print(correct_top3_count)
        if test_label in closest_3_labels:
            correct_top3_count += 1
        # print(correct_top3_count)

    top3_accuracy_value = correct_top3_count / total_tests * 100
    return top3_accuracy_value


In [ ]:
top3_acc = calculate_top3_accuracy(test_embeddings_dict, first_embeddings_dict)
print(f"Top-3 Accuracy: {top3_acc:.2f}%")